In [1]:
import requests
import time
from bs4 import BeautifulSoup
import json
import csv
import random
import pandas as pd
import re
import os
import urllib.parse
import urllib.request
import json

In [17]:
df = pd.read_csv("old_house_info.csv") 
   
# output the dataframe
df

,sub_url,build_year,house_name,address,price,district,id
0,/loupan/1037.html,2003,银石雅园,香洲区珠海九州大道西,26658,1,1037
1,/loupan/1361.html,2009,岭秀城,香洲区岭南路8号、公,25440,1,1361
2,/loupan/324.html,2002,粤海国际花园,香洲区拱北国防路1,29024,1,324
3,/loupan/319.html,2006,钰海山庄,香洲区前山兰埔路15,29924,1,319
4,/loupan/1339.html,2010,格力香樟,香洲区九洲大道西16,35414,1,1339
...,...,...,...,...,...,...,...
2496,/loupan/43429.html,2020,森宇阳光公寓,高栏港经济区平沙镇华,0,14,43429
2497,/loupan/43480.html,1995,西横街,,0,14,43480
2498,/loupan/4127.html,0,联港工业区,,0,14,4127
2499,/loupan/43251.html,1995,金盛花园,高栏港平沙镇美平东路,7925,14,43251


In [29]:
df = pd.read_csv("new_house_detail.csv") 
   
# output the dataframe
df

,developer,sur_trafic,sur_school,sur_environment,sur_business,sur_entertain,peoperty_fee,property_company,id
0,珠海世茂新领域房地产开发有限公司,一，形成1桥2通道3口岸3港口5机场的人流变现环。其中2通道为深中通道与深珠通道，目前深中通...,NaN,从整个岛的控规来看，近 200 万平方米的全岛上，除了三地联检大楼、岛内绿地跟停车区域、公建...,10 万方商业：为阶梯式 5 层的购物中心，将世茂集团商业战略合作的资源引进到，比如进驻到上...,世界级滨海公园：项目滨海景观轴为新月形状的映月公园，映月公园作为景观主轴，依次规划有逐浪栈道...,6.0元/㎡,世茂商娱物业,2040
1,珠海市耀君房地产开发有限公司,1、三横两纵多维路交通网络（三横：梅华路、人??路、九洲大道；两纵：情侣路、迎宾南路），逾2...,北岭小学、省一级初中国家级绿色学校拱北中学,5-10分钟步行到达珠海首个开放式商业街区富华里、拱北首个网红主题购物公园玖洲道以及购物中心...,1、商业配套：5-10分钟步行到达珠海首个开放式商业街区富华里、拱北首个网红主题购物公园玖洲...,项目北眺板樟山森林公园，绿化森林植被覆盖率高达80%，适合健步、锻炼及观光旅行。距国家4A级...,6.0元/㎡,广东光集团物业管理有限公司（一级资质,2096
2,珠海市美盈投资管理有限公司,项目直达拱北口岸、广珠轻轨前山站、终点站及港珠澳大桥出入口，项目属于轻轨及港珠澳大桥双重价值...,侨光小学、夏湾小学、十六小、十一中、夏湾中学,前山河边、市政公园，世纪城、红树湾等高端住宅小区,珠澳湾世纪中心，卓凡中心，宝莱康，拱北口岸关口/华润万家/明珠商业广场，天虹商场，富华里/摩...,圆明新园，炮台山公园，铂尔曼酒店，来魅力假日酒店，项目自带商业，餐饮，购物，健身房，万枫酒店...,8.0元/㎡,华发物业,1276
3,仁恒置业,情侣南路 拱北口岸总站2路4 路 9路 99 路 观光线 拱北口岸,夏湾小学、夏湾中学、珠海一中附属实验中学、拱北小学、拱北中学、北岭小学,拱北口岸核心资产，零距离珠澳双城生活圈，周边商圈环伺，交通便利，星级酒店林立，国宾级会客厅、...,拱北口岸商场，华润万家大型超市，沃尔玛超市，米兰百货,铂金洲仔酒店 来魅力。酒店：益健、新海利、棕泉、来魅力、东方印象、帝澜思、昌安等。,7.0元/㎡,仁恒物业,1246
4,凤凰泰富,"4路, 9路, 13路, 36路 99路公交车在粤海东站",拱北小学、拱北中学,"凤凰谷项目南侧为联安路，东连水湾路及情侣南路，西接迎宾南路。可乘坐4路, 9路, 13路, ...",万佳百货、米兰百货、免税商场、莲花路商业步行街,酒吧街、皇家台球俱乐部、粤海保龄球馆,6.0元/㎡,深圳彩生活,55
...,...,...,...,...,...,...,...,...,...
272,珠海高盛房地产开发公司,项目以南经珠海大道与港珠澳大桥西沿线相连，可快速通达港澳。项目周边西部沿海高速、高栏港高速等...,项目西侧紧邻平沙中心幼儿园（市一级幼儿园）、平沙实验小学（省一级学校），项目北侧珠海一中平沙...,优质高中、初中、小学、幼儿园等环伺项目左右，周边有医院、市场、酒店，大型商业和特色商街，旅游...,项目东侧紧邻钰海万国汇购物中心建筑3万㎡分为地上四层地下一层，包含国际影城，超市电影院等。文...,海泉湾度假区，南国游艇俱乐部，平沙影视城， 荷包岛，飞沙滩，孖髻山公园，台创园,2.8元/㎡,珠海高盛物业管理有限公司,2293
273,珠海市豪浚房地产开发有限公司,30 分钟速达区：依托鹤港?速、珠海大道、平东大道等城市大道，20 分钟瞬达机场，30 分...,优质学府，且项?为平沙新城学校学区房，周边覆盖 12 班幼?园、2 所学校、4 所?学、3 所中学,平沙新城少有的?城社区，29 万㎡纯粹亲海住区，配备中央泳池、全龄段休闲活动中?、绿氧双跑道...,邻里中心，华发商都等生活活配套,2.07 平方公里平沙生态湿地公园、43 万平公里温泉公园、17 公里沿海景观带、13 座景...,3.0元/㎡,天誉物业,2053
274,格力房产,公车204路、501路、502路、608路、K7路、K8路在平沙邮局站下。,幼儿园：时代幼儿园中小学：华丰小学，实验小学,NaN,华润万家,NaN,2.8元/㎡,格力物业,49
275,三一（珠海）置业有限公司,项目位置紧邻高栏港高速，往西则与西部沿海高速和黄茅海大桥对接，可通达珠三角各地，与珠三角地区...,平沙新城教育配套规划有3所小学、3所中学（初中）。我们项目南侧就是平沙新城学校，学校总用地面...,高栏港平沙新城是港产城融合标杆，拥有9大产业集中群，49条航线和5大码头，真正意义上的港珠澳...,平沙镇的文德广场、钰海国际广场，填补了高栏港区商业的空白。在项目旁边的邻里中心，用地面积约2...,平沙镇的文德广场、钰海国际广场，填补了高栏港区商业的空白。在项目旁边的邻里中心，用地面积约2...,3.0元/㎡,竹胜园物业,2226


In [22]:
df = pd.read_csv("old_house_detail.csv") 
   
# output the dataframe
df

,photo,room_num,area,orientation,price,status,h_num,s_num
0,https://img.0756fang.com/Upload/Fang/2021-11/s...,2房2厅1卫,73.23 ㎡,南,210,0,0,1037
1,https://img.0756fang.com/Upload/Fang/2023-03/2...,3房2厅1卫,96.43 ㎡,北,245,0,1,1037
2,https://img.0756fang.com/Upload/Fang/2022-02/s...,3房2厅2卫,105.07 ㎡,北,213,0,2,1037
3,https://img.0756fang.com/Upload/Fang/2020-07/s...,5房3厅3卫,255.92 ㎡,南北通,500,0,3,1037
4,https://img.0756fang.com/Upload/Fang/2022-09/2...,2房2厅1卫,79.88 ㎡,东南,212,0,4,1037
...,...,...,...,...,...,...,...,...
506,https://img.0756fang.com/Upload/Fang/2021-08/2...,4房2厅2卫,132.47 ㎡,4500,1,15,251,南
507,https://img.0756fang.com/Upload/Fang/2017-12/d...,3房2厅1卫,80 ㎡,3800,1,16,251,北
508,https://img.0756fang.com/Upload/Fang/2021-07/s...,3房2厅2卫,120 ㎡,3000,1,17,251,西
509,https://img.0756fang.com/Upload/Fang/2020-11/s...,4房2厅2卫,142.39 ㎡,3800,1,18,251,北


In [ ]:
df = pd.read_csv("new_house_type_info.csv") 
   
# output the dataframe
df

In [ ]:
df = pd.read_csv("old_house_type_info.csv") 
   
# output the dataframe
df

# crawler for new subdivision info

In [4]:
def new_sub_info():
    page = [2,1,2,1,2,1,1,4,2,3,4,6,7,2]
    for i in range(1,15):
        for j in range(0,page[i-1]):
            if(page[i-1] == 1):
                target = 'https://www.okhome.com/newhouse/q{}?sortIcon=Cell'.format(i)
            else:
                if(j+1 == 1):
                    target = 'https://www.okhome.com/newhouse/q{}?sortIcon=Cell'.format(i)
                else:
                    target = 'https://www.okhome.com/newhouse/p{}-q{}?sortIcon=Cell'.format(j+1,i)

            head={
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
            }

            resp = requests.get(url=target,headers=head)
            content=resp.text
            print(resp) 

            #begin to write
            obj = re.compile(r'class="house_card" href="(?P<sub_url>.*?)" target.*?<img src="(?P<photo>.*?)" alt="house_img">.*?class="house_name">(?P<house_name>.*?)</div>.*?<span>(?P<house_status>.*?)</span>.*?<div class=".*?">(?P<house_type>.*?)</div>.*?"text1">.*? (?P<address>.*?)</div>.*?"price">(?P<price>.*?)<span',re.S)

            result = obj.finditer(content)

            if os.path.exists('new_house_info.csv'):
                w = open("new_house_info.csv",mode="a+",newline='',encoding='utf-8')
                csvwriter = csv.writer(w)
            else:
                w = open("new_house_info.csv",mode="w",newline='',encoding='utf-8') #在这个地方必须要加上utf-8！！
                csvwriter = csv.writer(w)
                header = ("sub_url","photo","house_name","house_status","house_type","address","price","district","id")
                csvwriter.writerow(header)

            for it in result:
                dic=it.groupdict() #把每个变量整合到一起变成dict

                dic["district"] = i

                dic["price"] = "".join(filter(str.isalnum, dic["price"]))

                dic["id"] = "".join(filter(str.isdigit, dic["sub_url"]))
                
                #print(dic)

                csvwriter.writerow(dic.values())

            print("over!")

In [5]:
new_sub_info()

<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!


# crawler for new subdivsion detail info

In [40]:
def new_sub_detail():
    detail=pd.read_csv("new_house_info.csv",header=None)

    for row in range(1,281):
        sub_url = detail.iloc[row][0] #for sub page
        target = 'https://zh.okhome.com' + sub_url + 'xinxi.html'

        head={
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
        }

        resp = requests.get(url=target,headers=head)
        print(resp)
        content = resp.text.encode("iso-8859-1").decode('utf-8')

        #get info
        obj = re.compile(r'<div class="house_information_subitem_title">开  发  商</div>.*?subitem_main">(?P<developer>.*?)</div>.*?<div class="house_information_subitem_title">周边交通</div>.*?subitem_main">(?P<sur_trafic>.*?)</div>.*?subitem_main">(?P<sur_school>.*?)</div>.*?subitem_main">(?P<sur_environment>.*?)</div>.*?subitem_main">(?P<sur_business>.*?)</div>.*?subitem_main">(?P<sur_entertain>.*?)</div>.*?<div class="house_information_subitem_title">物 业 费</div>.*?subitem_main">(?P<property_fee>.*?)</div>.*?<div class="house_information_subitem_title">物业公司</div>.*?subitem_main">(?P<propety_company>.*?)</div>',re.S)
        result = obj.finditer(content)

        if os.path.exists('new_house_detail.csv'):
            w = open("new_house_detail.csv",mode="a+",newline='',encoding='utf-8')
            csvwriter = csv.writer(w)
        else:
            w = open("new_house_detail.csv",mode="w",newline='',encoding='utf-8') #在这个地方必须要加上utf-8！！
            csvwriter = csv.writer(w)
            header = ("developer","sur_trafic","sur_school","sur_environment","sur_business","sur_entertain","peoperty_fee","property_company","id")
            csvwriter.writerow(header)

        for it in result:
            dic=it.groupdict() #把每个变量整合到一起变成dict

            dic["sur_trafic"]=dic["sur_trafic"].replace("\n","")
            dic["sur_trafic"]=dic["sur_trafic"].replace("\r","")

            dic["sur_school"]=dic["sur_school"].replace("\n","")
            dic["sur_school"]=dic["sur_school"].replace("\r","")

            dic["sur_environment"]=dic["sur_environment"].replace("\n","")
            dic["sur_environment"]=dic["sur_environment"].replace("\r","")

            dic["sur_business"]=dic["sur_business"].replace("\n","")
            dic["sur_business"]=dic["sur_business"].replace("\r","")

            dic["sur_entertain"]=dic["sur_entertain"].replace("\n","")
            dic["sur_entertain"]=dic["sur_entertain"].replace("\r","")


            dic["id"] = detail.iloc[row][8] 

            print(dic)

            csvwriter.writerow(dic.values())

    resp.close()

In [41]:
new_sub_detail()

<Response [200]>
{'developer': '珠海世茂新领域房地产开发有限公司', 'sur_trafic': '一，形成1桥2通道3口岸3港口5机场的人流变现环。其中2通道为深中通道与深珠通道，目前深中通道已经建设当中，预计2024年即可通车。深珠通道东起深圳前海，西至珠海淇澳岛，横跨伶仃洋。目前该通道正在规划当中。前海段以西丽站为主要枢纽的规划已在推进当中二，项目内部约1万平方米 7.5 米层高。地上设有公共交通枢纽中心，同步在申请增设湾区 5 大机场候机大厅，项目地下负一层与整个口岸社会车辆停车场相', 'sur_school': '', 'sur_environment': '从整个岛的控规来看，近 200 万平方米的全岛上，除了三地联检大楼、岛内绿地跟停车区域、公建之外，唯一能够进行开发建设并可对外出售的仅有世茂港珠澳口岸城项目。接驳港珠澳大桥，与世界最大口岸相连接，紧密连接两岸三地', 'sur_buisiness': '10 万方商业：为阶梯式 5 层的购物中心，将世茂集团商业战略合作的资源引进到，比如进驻到上海世茂广场的知名商家，同时更以港澳居民日常衣食住行相关的主流品牌定向招商，打造一个犹如置身港澳的 24小时不夜城.五星品牌酒店：项目将引进5家品牌酒店，目前初步确定的为凡象酒店、茂御酒店、茂御居、睿选等,也同步在跟国际奢华酒店接洽当中。  意向引进像洲际同等品牌的全球五星级酒店，以港澳人比较熟知的文华东', 'sur_entertain': '世界级滨海公园：项目滨海景观轴为新月形状的映月公园，映月公园作为景观主轴，依次规划有逐浪栈道、海上剧场、极光喷泉、游艇码  头、望海眺台等。', 'property_fee': '6.0元/㎡', 'propety_company': '世茂商娱物业', 'id': '2040'}
<Response [200]>
{'developer': '珠海市耀君房地产开发有限公司', 'sur_trafic': '1、三横两纵多维路交通网络（三横：梅华路、人??路、九洲大道；两纵：情侣路、迎宾南路），逾20条公交线路，通往珠海各个方向。2、3分钟?程到达拱北口岸，10分钟通关瞬达澳?。3、广珠轻轨一期工程珠海站至横琴口岸现已在运行当中，横琴口岸是唯一24小时通关口岸，未来更是可以40分钟到达珠海

<Response [200]>
{'developer': '世纪都市', 'sur_trafic': '公交路线：3路、3A、10路、10A、7路、68路、69路至美丽湾站；17路、32路至神前总站', 'sur_school': '教育：凤凰海域国际幼儿园、香洲区十八小、珠海市第七中学、珠海市第三中学等全龄教育，给孩子成长提供优质的成长空间；教育：凤凰海域国际幼儿园、香洲区十八小、珠海市第七中学、珠海市第三中学等全龄教育，给孩子成长提供优质的成长空间；教育：凤凰海域国际幼儿园、香洲区十八小、珠海市第七中学、珠海市第三中学等全龄教育，给孩子成长提供优质的成长空间；', 'sur_environment': '凤凰公馆位于香洲区，香洲区属于南海之滨、珠江口西岸，隔海东望香港，南面陆路接壤澳门，背倚经济发达的珠江三角洲腹地，是广东省珠海市中心城区，也是珠海市政治、经济、文化、交通和金融中心。从本项目出发约10分钟到达扬名广场商圈，畅享日月贝歌剧院、野狸岛、扬名广场等生活体验，以及香洲港码头，可乘船领略珠海各大岛屿风光。', 'sur_buisiness': '商业：小区楼下200米即有美丽湾市场，可满足日常吃喝玩乐的生活需求，出门10分钟速达扬名广场、茂业百货、丹田城市广场及其他休闲设施于一体的商业综合体；', 'sur_entertain': '总投资超过13亿中国唯一建在海岛上的歌剧院日月贝歌剧院、投资10亿的珠海博物馆、展览馆、艺术中心等等这些市政重大项目，从项目出发5分钟内触手可得；休闲：顶级品牌酒店——悦椿度假酒店、天然氧吧——凤凰山森林公园、珠海新象征——凤凰塔，另有凤凰湾，情侣路，美丽湾驿站，海天驿站，海天公园等，都是您跟家人日常放松、亲近感情的好去处。', 'property_fee': '0.0元/㎡', 'propety_company': '珠海都市物业管理有限公司', 'id': '2055'}
<Response [200]>
{'developer': '珠海景泽房产公司', 'sur_trafic': '美丽湾   神前总站17路.32路.3路.3A路.10路.10A路.68路.69路', 'sur_school': '珠海三中  七中，凤凰海域国际幼儿园，第十八小学。', 'sur_environment': '海天公园  凤凰山  

<Response [200]>
{'developer': '山东颐养健康置业集团有限公司', 'sur_trafic': '明珠路、人民西路、翠微西路等城市主干道环绕，15分钟经广澳高速、广珠西线、金琴快线畅行大湾区各市，毗邻广珠轻轨明珠站，链接广州南站通达全国', 'sur_school': '香府一品择此一方宝地，延续学风，打造1所小学、2所幼儿园，构建全龄教育大城。翠微小学，第十中学，华昌小学，嘉园幼儿园，福来幼儿园，健华医院、上冲医院、珠海嘉济医院、兰埔门诊部、香洲区人民医院 前山电影院、前山游泳场、中山亭、新华书店等', 'sur_environment': '上冲医院、香洲区人民医院、珠海市人民医院、妇幼保健院、中山大学附属第五医院等', 'sur_buisiness': '沃尔玛、山姆会员店、诚丰广场、世邦家居世界、富华里、玖洲道、华发商都、优特汇、海韵城等，全部15分钟畅达', 'sur_entertain': '珠海市体育中心，梅华城市公园、板障山森林公园、圆明新园、梦幻水城、香山湖公园、上冲/翠景社区公园、海天公园等', 'property_fee': '3.0元/㎡', 'propety_company': '奥园智慧生活服务', 'id': '2169'}
<Response [200]>
{'developer': '珠海市富景居投资有限公司', 'sur_trafic': '可乘坐6路、16路、31路、33路、40路、43路、55路、99路、观光巴士至城轨明珠站下车即可；附近百米内有上冲长途汽车客运站和城轨明珠站。', 'sur_school': '香洲第十七小学，新世纪实验学校，希望之星实验学校，香洲第十二小学，立才学校，恒隆学校，上冲中学，九中幼儿园：翠华幼儿园、小慧星幼儿园', 'sur_environment': '凤山府为珠海市政府重点工程配套项目，并与广珠城轨、珠海市长途汽车站及珠海市公共汽车总站配套，身处交通中央，三站一中心，是名副其实的交通核心地带。', 'sur_buisiness': '小区内部三层商业/家乐福、中影电影院等', 'sur_entertain': '凤山府同时地处繁华商业中心，周边各种丰富商业成熟，文化气息浓厚。', 'property_fee': '2.9元/㎡', 'propety_compan

<Response [200]>
{'developer': '珠海保通投资有限公司', 'sur_trafic': '拱北轻轨站，拱北汽车总站，公交车站都在周边', 'sur_school': '夏湾小学、夏湾中学、珠海一中附属实验中学、拱北小学、拱北中学、北岭小学', 'sur_environment': '拱北口岸核心资产，零距离珠澳双城生活圈，周边商圈环伺，交通便利，星级酒店林立，国宾级会客厅、商务会馆、等资源随时共享。', 'sur_buisiness': '拱北口岸商场，华润万家大型超市，沃尔玛超市，米兰百货', 'sur_entertain': '酒店：益健、新海利、棕泉、来魅力、东方印象、帝澜思、昌安等。', 'property_fee': '5.0元/㎡', 'propety_company': '佳兆业', 'id': '1190'}
<Response [200]>
{'developer': '珠海市峻景房地产开发有限公司', 'sur_trafic': '广珠轻轨拱北站，过关澳门拱北口岸，公交路线82，62，35，k10，k8……', 'sur_school': '侨光小学，夏湾小学，夏湾中学…', 'sur_environment': '前山河边，楼下市政公园，边上有华发国际花园，华发世纪城，中信红树湾等高档小区，还有在建五星级史莱康酒店等', 'sur_buisiness': '夏湾批发市场，世纪城华润万家，华发商都，富华里商圈，拱北口岸购物商城。华发卓凡商业中心，粤华路、侨光西路成熟商业配套；500米内有中国银行、建设银行、交通银行、平安银行、邮政银行等', 'sur_entertain': '前山河前市政公园，竹仙洞公园，高尔夫球场，湾仔旅游码头，世纪城酒吧街，汉庭唛潮ktv……', 'property_fee': '5.0/㎡', 'propety_company': '龙光物业', 'id': '117'}
<Response [200]>
{'developer': '珠海正方集团', 'sur_trafic': '【1大桥】港珠澳大桥【2高速】广澳高速 / 京港澳高速【2跨海通道】深中通道（建设中） / 深珠通道（规划）【2城市快线】金琴快线 / 兴业快线枢纽【2国际机场】澳门国际机场 / 珠海金湾机场【4轨道交汇】广州地铁

<Response [200]>
{'developer': '珠海市润地房地产有限公司', 'sur_trafic': '粤港澳大湾区深珠通道落脚点；距离广珠城际轻轨唐家北站700米、距离珠海情侣路金边海岸900米；项目出发30分钟通过深珠通道抵达深珠前海；项目出发20分钟通过金琴快线北延长线到达深中大桥；项目出发20分钟通过金琴快线、兴业快线通往珠海老城区\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t粤港澳大湾区深珠通道落脚点；距离广珠城际轻轨唐家北站700米、距离珠海情侣路金边海岸900米；项目出发30分钟通过', 'sur_school': '一站式全龄教育 5幼3小2中1高4大学：中山大学、北理工、北师大、UIC、礼和中学（在建）、金鼎第一小学、礼和小学 、唐家湾幼儿园等，学风优良醇厚，孩子成长无忧；更自带高端幼儿园，成长第一站即享优渥。', 'sur_environment': '登年轻力 约4万㎡不打烊潮流商街 建面约20-200㎡潮流商街24+1小时的精彩：港式早茶、深夜食堂、萌宠、时尚买手店等新锐生活方式集合街区，小酒吧、茶坊、西餐厅、网红美食……繁华自成，每天多出一小时，享受不打烊的都会精彩。与商业MALL互补共赢：近邻商业MALL，名副其实的消费旺场，商家和消费者共享繁华盛景，实现多方共赢。', 'sur_buisiness': '繁华年轻力 约8万㎡全业态购物中心 宝龙商业，华润打造润街100%自持', 'sur_entertain': '生活配套：项目1km有菜肉市场和一级邻里中心市场，同时自带10万㎡全时区商业旗舰（MALL+商业步行街）,足以满足您得日常生活所需。休闲娱乐配套：国际赛车场、翠湖高尔夫球场、淇澳岛湿地公园及唐家湾古镇。6大公园环伺：咫尺绿谷中轴景观带、约1.5公里即达珠海名片情侣北路；半圆之园公园、湿地公园等，尽享都市森呼吸绿生活。周边医院：周边有中大五院、珠海高新区人民医院，明年在南方软件园附近修建', 'property_fee': '3.0元/㎡', 'propety_company': '华润物业', 'id': '2258'}
<Response [200]>
{'developer': '美的置业', 'sur_trafic': 'OD智能城市综合体 脉通大湾区公交——

...

<Response [200]>
{'developer': '珠海市裕沙房地产开发有限公司', 'sur_trafic': '803,Z125。Z108', 'sur_school': '天姿幼儿园，三灶中心小学，三灶中学。', 'sur_environment': '列圣山森林公园.伟民广场.唐人街市场', 'sur_buisiness': '唐人街市场，嘉珠时代广场', 'sur_entertain': '列圣山森林公园.伟民广场.唐人街市场，嘉珠时代广场', 'property_fee': '3.0/㎡', 'propety_company': '时代物业', 'id': '26'}
<Response [200]>
{'developer': '个人产权', 'sur_trafic': '往市区：交通：201、K9（香洲）、504（井岸）、801、803、      （三灶环线）Z101、Z104、Z108、Z125', 'sur_school': '三灶中心小学、三灶中学、吉林大学珠海校区、遵义医学院珠海校区、珠海市职业技术学院、珠海市实验中学高中部、', 'sur_environment': '位于三灶最繁华地区，周边配套齐全，生活非常便利。', 'sur_buisiness': '唐人街市场、得一超市、万家美购物广场、麦当劳、肯德基（益百家）、红叶美食广场、鹏泰购物广场、中影傲视电影城、三灶文化中心、格林豪泰酒店、七天连锁酒店、四大银行、银柜量版式KTV、鸿银之星卡拉OK厅等', 'sur_entertain': '唐人街市场、得一超市、万家美购物广场、麦当劳、肯德基（益百家）、红叶美食广场、鹏泰购物广场、中影傲视电影城、三灶文化中心、格林豪泰酒店、七天连锁酒店、四大银行、银柜量版式KTV、鸿银之星卡拉OK厅等', 'property_fee': '0.0/㎡', 'propety_company': '', 'id': '61'}
<Response [200]>
{'developer': '珠海拓茂房地产开发有限公司', 'sur_trafic': '珠海大桥、香海大桥（在建）、洪鹤大桥（在建）、金海大桥（在建）、广佛江珠轻轨（规划）', 'sur_school': '建发幼儿园（在建）、金茂幼儿园（规划中）、中铁建幼儿园（规划中）、金茂小学（

<Response [200]>
{'developer': '珠海颐泽企业管理有限公司', 'sur_trafic': '一桥主城立体交通畅行湾区招商华发湖心明荟坐拥湖心路、双湖路双主轴，雄踞珠海大桥、香海大桥（预计2022年通车）桥头堡', 'sur_school': '乐思国际幼儿园、齐正小学、城南中学、红旗中学，规划12班幼儿园，24班小学，30班初中', 'sur_environment': '西城之芯、5分钟繁华配套圈32公顷湿地公园、8大主题景观', 'sur_buisiness': '家和城商业广场', 'sur_entertain': '32公顷湿地公园、8大主题景观8万平星空公园', 'property_fee': '3.0元/㎡', 'propety_company': '海伦堡物业管理有限公司', 'id': '2251'}
<Response [200]>
{'developer': '珠海市恒基房产开发有限公司', 'sur_trafic': '交通公交：406路、407路、419路（富强中站）、416路（坚士制锁站），沿富强十路直行150米至项目入口，右转直行到达项目自驾：珠峰大道或西部沿海高速——高栏港高速辅路——富强十路——五洲富山园项目', 'sur_school': '乾务镇第二中心幼儿园、三村幼儿园、新村幼儿园、五山中心幼儿园中小学乾务五山中心小学、五山中学', 'sur_environment': '自建2.2万平商业综合体，米夏村市场、马山市场御温泉度假区、海泉湾度假区、斗门古街、金台寺、市政公园（建设中）珠海市第五人民医院，斗门富山门诊部', 'sur_buisiness': '自建2.2万平商业综合体，米夏村市场、马山市场', 'sur_entertain': '五洲富山园，打造富山片区稀缺高品质住宅、商业小区，满足居民商务、居住、购物、娱乐需求。五洲富山园园林风格定义为现代主义风格，设计上重视空间格局，彰显都市感和现代感，打造简约淡雅、自然清新的居住环境。配件2.2万㎡一站式商业，高端购物商场、品牌餐饮、娱乐、休闲等精彩汇聚，幅射周边8公里，覆盖消费人群过万。并精心打造数万平米运动休闲湿地公园，全龄居住社区，包含儿童活动、羽毛球场、篮球场、青年活动、老年', 'property_fee': '2.0元/㎡', 'pr

<Response [200]>
{'developer': '珠海依云房地产有限公司', 'sur_trafic': '项目沿河段交通路况已基本完善，通车在即，项目东侧为成裕巴士站和益力集团巴士站（面前正在洽谈更名为招商依云水岸巴士站），经过该站的公交车线路为306路（白蕉长途客运站——北澳），前往本项目可在成裕巴士站下车。珠海市区方向：乘坐603、601、609、602、K5、K4、K6等在鳄鱼岛下车，步行2分钟到尖峰桥东站乘坐306公交至成裕巴士站，前行100米井岸方向：乘坐306公交至成裕巴士站，前行', 'sur_school': '霞山公园、尖峰山公园、一河两岸八大公园配套（凤山公园、扬帆公园、河畔广场、裕泽海滨公园、云峰公园、乔园、黄杨休闲广场、叠翠公园）', 'sur_environment': '港珠澳大桥成就世界珠海，内陆唯一与港澳同时连接的城市；位处西部中心城区的核心位置，百年央企打造全国第三', 'sur_buisiness': '华润万家已进驻招商', 'sur_entertain': '配 套内  容小区内配套 商业街、休闲广场、儿童活动区、健身区、社区泳池教育\t蓓蕾艺术幼儿园、白蕉镇中心幼儿园、成裕围小学、白蕉中心小学、联厂小学、城东中学、田家炳中学、井岸青少年宫购物/超市\t自身商业街、华润万家、百惠购物中心、千惠百货、永发优惠商城、华美超市、井岸步行街、大信新都汇（在建）市场\t金山市场、白蕉中心市场、城东市场、东湖市场、白藤湖市场医院\t遵义五院、侨立中医', 'property_fee': '2.6元/㎡', 'propety_company': '招商局物业管理有限公司珠海分公司', 'id': '99'}
<Response [200]>
{'developer': '珠海中邦房地产开发有限公司', 'sur_trafic': '304路、306路、307路、603路。520路', 'sur_school': '幼儿园：刘诗昆音乐幼儿园中小学： 城东小学、田家炳中学、实验中学、区二中 大学：城市职业技术学院、广东科学', 'sur_environment': '医院：遵医第五院邮局：邮政银行：中国银行、华润银行、建设银行、工商银行 农商银行 其他：市场 华润万家 华海鹏程五星级酒店', 'sur_buisiness': '

<Response [200]>
{'developer': '珠海市锦荣房地产开发有限公司', 'sur_trafic': '301、503、504、601、601B、602、602A、609、K4、K5路公交车香海彼岸巴士站下', 'sur_school': '齐正小学、城南中学、科技学院、吉林大学', 'sur_environment': '约2万㎡大型生活休闲商业街区，省级奢华游泳池，标准篮球场，儿童活动广场等。', 'sur_buisiness': '新万家福超市、美宜家、华润万家、万家美超市', 'sur_entertain': '白藤山公园、白藤湖度假村、鳄鱼岛、御温泉、金湾高尔夫', 'property_fee': '3.0元/㎡', 'propety_company': '时代物业', 'id': '2145'}
<Response [200]>
{'developer': '珠海市斗门汇业房地产开发商有限公司', 'sur_trafic': '交通可乘坐603、601、601B、602、602A、609、504、301、503', 'sur_school': '白藤湖中学、白藤湖中心小学 幼儿园、颖星幼儿园(白藤三路) 、西江月幼儿园', 'sur_environment': '本项目自带45000平方米，拥有低容积率以及高使用率。项目背靠斗门白藤湖，可足不出户享受一线湖景，西湖度假村，空气清新，环境优美。', 'sur_buisiness': '隔壁有家和城，大型集中商业，涵盖超市、影院、百货等多种业态，规模逾三万平米，华润万家强势进驻! 小区自带4.5万平商业! 购物就在家门口', 'sur_entertain': '白藤山公园、白藤湖度假村、鳄鱼岛、御温泉、金湾高尔夫', 'property_fee': '3.0元/㎡', 'propety_company': '珠海怡家园', 'id': '1312'}
<Response [200]>
{'developer': '金地集团', 'sur_trafic': '公交站304路、306路、307路、603路；白蕉长途客运站；井岸中心站', 'sur_school': '田家炳中学、城东中学、成裕围小学 、联厂小学、东和小学、青少年宫、蓓蕾艺术幼儿园；', 'sur_environment':

<Response [200]>
{'developer': '广拓置业', 'sur_trafic': '京华假日湾公交站（826-2路、812路、k30、Z142路等）', 'sur_school': '15年学区教育：珠海一中平沙校区（国家示范性高中）、平沙第一中学（市一级学校）、平沙实验小学（省一级学校）、平沙中心幼儿园（市一级幼儿园）等幼、小、初、高全龄化学校近在咫尺', 'sur_environment': '高栏港区后花园，政府重视，发展快，经济辐射大；紧邻平沙主线路升平大道，坐拥平沙繁华中心', 'sur_buisiness': '京华MALL,时代港商荟，台湾风情街，钰海国际广场，文德广场', 'sur_entertain': '海泉湾度假区，南国游艇俱乐部，平沙影视城，台湾风情街，荷包岛，飞沙滩，孖髻山公园，台创园', 'property_fee': '2.7元/㎡', 'propety_company': '金地', 'id': '1203'}
<Response [200]>
{'developer': '珠海汇华星城置业有限公司', 'sur_trafic': '项目位置紧邻高栏港高速，往西则与西部沿海高速和黄茅海大桥对接，可通达珠三角各地，与珠三角地区形成1小时经济圈；往南可接驳珠海大道，经珠海大道可与港珠澳大桥西沿线相接，可快速通达珠海市区及港澳', 'sur_school': '平沙新城教育配套规划有3所小学、3所中学（初中）。项目南侧就是平沙新城学校，学校总用地面积约3.7万㎡，建筑面积约3万㎡，预计设置36个班（公办学位），九年一贯制，2022年9月即可建成使用，真正意义上的教育就在家门口。', 'sur_environment': '高栏港平沙新城是港产城融合标杆，拥有9大产业集中群，49条航线和5大码头，真正意义上的港珠澳价格洼地，下一个宜居新城和产业投资热点', 'sur_buisiness': '平沙镇的文德广场、钰海国际广场，填补了高栏港区商业的空白。在项目旁边的邻里中心，用地面积约2.5万㎡，未来也可满足业主的生活需求，预计2020年上半年开工建设，2021年建成。', 'sur_entertain': '平沙镇的文德广场、钰海国际广场，填补了高栏港区商业的空白。在项目旁边的邻里中心，用地面积约2.5万㎡，未来也可满足

# crawler for new house type info

In [6]:
def new_house_type():
    detail=pd.read_csv("new_house_info.csv",header=None)

    for row in range(1,281):
        sub_url = detail.iloc[row][0] #for sub page
        target = 'https://zh.okhome.com' + sub_url + 'huxing.html'

        head={
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
        }

        resp = requests.get(target,headers=head)
        content = resp.text.encode("iso-8859-1").decode('utf-8')
        soup = BeautifulSoup(content,"html.parser")

        type_info = []

        for i in soup.find_all(name="div", attrs={"class" : "swiper-container houseTypeImg_swiper_infor"}):
            #photo & name
            k=i.find_all("img")
            k=str(k)

            obj = re.compile(r'<img alt="(?P<name>.*?)" src="(?P<photo>.*?)"/>',re.S)
            result = obj.finditer(k)

            for it in result:
                dic=it.groupdict() #把每个变量整合到一起变成dict
                t = {}

                if(dic['name'] != ""):
                    t["photo"] = dic["photo"]
                    type_info.append(t)

            #print(type_info)

            j=i.find_all(name="div", attrs={"class" : "infor_item_value"})
            #j = list(j)
            #print(j)

            n = len(type_info)
            for g in range(n):
                type_info[g]["room_num"] = j[g*7].text
                type_info[g]["area"] = j[(g*7)+1].text
                type_info[g]["price"] = j[(g*7)+2].text
                
                type_info[g]["price"]=type_info[g]["price"].replace("\n","")
                type_info[g]["price"]=type_info[g]["price"].replace("\r","")
                type_info[g]["price"] = "".join(filter(str.isalnum, type_info[g]["price"]))
                
                type_info[g]["orientation"] = j[(g*7)+5].text
                type_info[g]["h_num"] = g
                type_info[g]["s_num"] = detail.iloc[row][8]

            #print(type_info)
            
            if os.path.exists('new_house_type_info.csv'):
                w = open("new_house_type_info.csv",mode="a+",newline='',encoding='utf-8')
                csvwriter = csv.writer(w)
            else:
                w = open("new_house_type_info.csv",mode="w",newline='',encoding='utf-8') #在这个地方必须要加上utf-8！！
                csvwriter = csv.writer(w)
                header = ("photo","room_num","area","price","orientation","h_num","s_num")
                csvwriter.writerow(header)
                
            for d in type_info:
                print(d.values())
                csvwriter.writerow(d.values())

    resp.close()

In [7]:
new_house_type()

dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-11/sD1FA3355A79D4B968AB029729CBE530F.jpg', '1室0厅0卫', '45.0㎡', '33333元', '南', 0, '2040'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-11/sA0DDBD7D4DB94A5695324098F3985D6A.jpg', '2室0厅0卫', '78.0㎡', '33333元', '东南', 1, '2040'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-11/s7C54DA30BAFB4E41A3C7C2822846A882.jpg', '1室0厅0卫', '40.0㎡', '32500元', '东', 2, '2040'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-11/sA2BA4DAA77304AC898C35B63EDF9A1DA.jpg', '1室0厅0卫', '41.0㎡', '34146元', '北', 3, '2040'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-12/sA247CB4DEE4E442AAB09F3CE3C09D879.jpg', '4室2厅2卫', '216.0㎡', '40277元', '东南', 4, '2040'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-12/s027601A992FF41DBB2D0AEBF936938C5.jpg', '2室2厅1卫', '109.0㎡', '38440元', '东', 5, '2040'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-12/sEAAAC439B2FB4906A9D9D2F7

dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-10/29e2cf0f-7364-47e0-ad7b-bb1108aca875.jpg', '4室2厅5卫', '218.0㎡', '35779元', '南北通', 0, '1339'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-10/55c5e5a7-04bb-4926-b9cc-9f0fb07e4995.jpg', '3室2厅3卫', '233.0㎡', '42918元', '南北通', 1, '1339'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-10/629e048d-f13f-4036-953e-6bd109c9fe77.jpg', '6室3厅4卫', '196.0㎡', '47346元', '南北通', 2, '1339'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-10/14338e32-64b0-469b-a681-5272a5ed5a99.jpg', '6室3厅4卫', '196.0㎡', '47346元', '南北通', 3, '1339'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-10/111f8b36-e7cf-4787-9ffb-f7310d6d4c19.jpg', '6室3厅4卫', '196.0㎡', '47346元', '南北通', 4, '1339'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-10/629e048d-f13f-4036-953e-6bd109c9fe77.jpg', '6室3厅4卫', '196.0㎡', '47346元', '南北通', 5, '1339'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2

dict_values(['https://img.0756fang.com/Upload/fyEstate/160410151910E9B06FDFF80F52E4E9A8.jpg', '0室0厅0卫', '0㎡', '0元', '', 0, '2254'])
dict_values(['https://img.0756fang.com/Upload/fyEstate/1604101520301711F399EC9D5C54F43F.jpg', '0室0厅0卫', '0㎡', '0元', '', 1, '2254'])
dict_values(['https://img.0756fang.com/Upload/fyEstate/1604101519493812DE52BFBDB5693D6A.jpg', '0室0厅0卫', '0㎡', '0元', '', 2, '2254'])
dict_values(['https://img.0756fang.com/Upload/fyEstate/16041015195975FC4399165D666389D7.jpg', '0室0厅0卫', '0㎡', '0元', '', 3, '2254'])
dict_values(['https://img.0756fang.com/Upload/fyEstate/160410152010B601DAE69758EB242D76.jpg', '0室0厅0卫', '0㎡', '0元', '', 4, '2254'])
dict_values(['https://img.0756fang.com/Upload/fyEstate/1604101520202C82B5DA3C63ED2DF01A.jpg', '0室0厅0卫', '0㎡', '0元', '', 5, '2254'])
dict_values(['https://img.0756fang.com/Upload/fyEstate/1604101520301711F399EC9D5C54F43F.jpg', '0室0厅0卫', '0㎡', '0元', '', 6, '2254'])
dict_values(['https://img.0756fang.com/Upload/fyEstate/1604101520410C6D8C6FE

dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-04/202204091250192408.jpg', '3室2厅2卫', '89.0㎡', '25280元', '西南', 0, '2248'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-04/202204091256186625.jpg', '4室2厅2卫', '99.0㎡', '26262元', '南', 1, '2248'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-04/202204090101352770.jpg', '3室2厅1卫', '79.0㎡', '25316元', '南', 2, '2248'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-10/sF6AD1CDC427C4290A8DBF6EC0077272B.jpeg', '3室2厅2卫', '89.3㎡', '26875元', '东南', 0, '70'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-10/sC9CA8FEB7AD5408F94BF189F1B7CBDFB.jpeg', '4室2厅2卫', '110.6㎡', '31193元', '东南', 1, '70'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-04/202204151010463278.jpg', '4室2厅3卫', '151.1㎡', '27796元', '南北通', 2, '70'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-10/202110220427298466.jpeg', '4室2厅3卫', '248.0㎡', '28225元', '南北通', 0, '1165'])
dict_values

dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-04/1EABBDEB650A4F4BAACEA6507AF6AE51.jpeg', '3室2厅2卫', '91.0㎡', '25824元', '南', 0, '2120'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-04/0B9AD20603914D13B52E0B49114A242C.jpeg', '3室2厅2卫', '139.9㎡', '22158元', '南北通', 1, '2120'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-10/202210141112355526.jpg', '1室1厅1卫', '15.0㎡', '66666元', '东', 0, '2279'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-10/202210141130016688.jpg', '1室1厅1卫', '30.0㎡', '33333元', '东', 1, '2279'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-10/202210141137433543.jpg', '1室1厅1卫', '26.3㎡', '22053元', '西南', 2, '2279'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-10/202210141135454211.jpg', '1室1厅1卫', '37.1㎡', '24258元', '东南', 3, '2279'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-10/202210141137433543.jpg', '1室1厅1卫', '56.8㎡', '23943元', '西南', 4, '2279'])
dict_values(

...

dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-12/d2fdb4c5-572c-42da-9120-229ae76bdd49.jpg', '3室2厅2卫', '92.0㎡', '25217元', '南', 0, '1353'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-12/d17eccf2-ab2e-4865-9dff-7c0a7194c0da.jpg', '3室2厅1卫', '80.0㎡', '25000元', '西南', 1, '1353'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-12/9b8ec805-7874-4d02-84a7-20a48550f325.jpg', '4室2厅2卫', '105.0㎡', '26190元', '南北通', 2, '1353'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-12/1d3f8b65-37d3-4ec4-a0bb-6f536030a593.jpg', '4室2厅2卫', '129.0㎡', '25968元', '南北通', 3, '1353'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-08/sBC30DDDFF3BD443199E2F0C472601982.jpeg', '4室2厅2卫', '121.0㎡', '21487元', '南北通', 0, '2029'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-08/sF961F53FAAAE46F58C50AA4A1636DBD7.jpeg', '3室2厅2卫', '90.3㎡', '21594元', '南北通', 1, '2029'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-08/sFE

dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-04/202204090224593410.jpeg', '3室2厅2卫', '97.0㎡', '18556元', '东南', 0, '2104'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-04/202204090225206226.jpeg', '4室2厅2卫', '119.0㎡', '18487元', '东南', 1, '2104'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-12/s23723B2D6EC249FA908A0E7C4EF5DB0C.png', '4室2厅2卫', '143.0㎡', '23076元', '南北通', 2, '2104'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-04/202204090224240533.jpeg', '4室2厅2卫', '127.0㎡', '18897元', '南北通', 3, '2104'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-05/2d432b19-9feb-4422-bfd1-c44b799b3b59.jpg', '3室2厅2卫', '90.0㎡', '12111元', '南北通', 0, '1289'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-05/25fb710e-2090-4f2c-b9a7-c750c72d613e.jpg', '3室2厅1卫', '99.0㎡', '11919元', '南北通', 1, '1289'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-01/s3F42F2C62F5D40CF9B75F8E883754575.jpg', '4室2厅2卫', '1

dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-04/sC34FA191B193428BAD3EA548EEFA9FCB.jpg', '2室2厅1卫', '68.0㎡', '16911元', '东', 0, '1259'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-04/sAA5DAE7AF95045CAA53449B13726573F.jpg', '3室2厅2卫', '89.0㎡', '15056元', '西南', 1, '1259'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-04/s76FD9B29664B494EA32FA1B374597495.jpg', '1室1厅1卫', '43.0㎡', '17209元', '东南', 2, '1259'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-04/s11C16C3CC5F3483DB543F646696C110A.jpg', '3室2厅1卫', '83.0㎡', '16024元', '西南', 3, '1259'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-08/d3e92377-3cc2-49e1-aec6-69f328817e6e.jpg', '3室2厅2卫', '89.0㎡', '20786元', '南', 0, '77'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-08/18ce9969-8a79-49ab-b872-f6898f3e20ad.jpg', '4室2厅2卫', '110.0㎡', '19818元', '南北通', 1, '77'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-03/C8541A9F920C461280150

dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-11/202111171028417718.jpg', '0室0厅0卫', '0.0㎡', '0元', '南', 0, '2225'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-11/202111171029566286.jpg', '0室0厅0卫', '0.0㎡', '0元', '南', 1, '2225'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-11/202111171031005373.jpg', '0室0厅0卫', '0.0㎡', '0元', '南', 2, '2225'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-11/202111171032178308.jpg', '0室0厅0卫', '0.0㎡', '0元', '南', 3, '2225'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-11/202111171033064373.jpg', '0室0厅0卫', '0.0㎡', '0元', '南', 4, '2225'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2021-11/202111171034291785.jpg', '0室0厅0卫', '0.0㎡', '0元', '北', 5, '2225'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-09/sCBE5927A868949E4964D53630282DF94.jpg', '2室2厅1卫', '72.0㎡', '0元', '西南', 0, '2061'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2020-09/s

dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-12/17eadc06-a364-445a-91e1-f78e66e2d720.jpg', '5室2厅3卫', '192.0㎡', '6770元', '南', 0, '1352'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-12/137a30c8-8513-41d3-ab76-5df4ff875c39.jpg', '4室2厅3卫', '168.0㎡', '8214元', '东北', 1, '1352'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2019-12/17eadc06-a364-445a-91e1-f78e66e2d720.jpg', '3室2厅2卫', '98.0㎡', '14081元', '西南', 2, '1352'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-12/202212261144182642.jpeg', '3室2厅2卫', '100.0㎡', '12000元', '南', 0, '2268'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-12/202212261143101151.jpeg', '2室2厅1卫', '79.0㎡', '12151元', '南', 1, '2268'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-12/202212261142260747.jpeg', '4室2厅2卫', '115.0㎡', '12347元', '南北通', 2, '2268'])
dict_values(['https://img.0756fang.com/Upload/Dictionary/2022-12/202212261146233427.jpeg', '3室2厅2卫', '103.0㎡', '12135元', 

# crawler for old sub info

In [1]:
def old_sub_info():
    page = [14,21,22,20,17,8,9,13,8,5,7,28,34,8]
    for i in range(1,15):
        for j in range(0,page[i-1]):
            if(page[i-1] == 1):
                target = 'https://www.okhome.com/house/q{}'.format(i)
            else:
                if(j+1 == 1):
                    target = 'https://www.okhome.com/house/q{}'.format(i)
                else:
                    target = 'https://www.okhome.com/house/p{}-q{}'.format(j+1,i)

            head={
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
            }

            resp = requests.get(url=target,headers=head)
            content=resp.text
            print(resp) 

            #begin to write
            obj = re.compile(r'<div class="neighbourhood_card2">.*?"(?P<sub_url>.*?)" class=".*?<img src="(?P<photo>.*?)" alt="">.*?<span class="year">(?P<build_year>.*?)</span>.*?neighbourhood_card_name">(?P<house_name>.*?)</a>.*?card_text">(?P<address>.*?)</div>.*?neighbourhood_card_price">(?P<price>.*?)<span',re.S)

            result = obj.finditer(content)

            if os.path.exists('old_house_info.csv'):
                w = open("old_house_info.csv",mode="a+",newline='',encoding='utf-8')
                csvwriter = csv.writer(w)
            else:
                w = open("old_house_info.csv",mode="w",newline='',encoding='utf-8') #在这个地方必须要加上utf-8！！
                csvwriter = csv.writer(w)
                header = ("sub_url","build_year","house_name","address","price","district","id")
                csvwriter.writerow(header)

            for it in result:
                dic=it.groupdict() #把每个变量整合到一起变成dict

                dic["district"] = i

                dic["price"] = "".join(filter(str.isalnum, dic["price"]))

                dic["id"] = "".join(filter(str.isdigit, dic["sub_url"]))
                
                #print(dic)

                csvwriter.writerow(dic.values())

            print("over!")

In [7]:
old_sub_info()

<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [200]>
over!
<Response [

# crawler for old subdivision detail info

In [21]:
def old_sub_detail():
    detail=pd.read_csv("old_house_info.csv",header=None)

    for row in range(1,2502):
        target = 'https://www.okhome.com' + detail.iloc[row][0]

        head={
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
        }

        resp = requests.get(url=target,headers=head)
        print(resp)
        content = resp.text.encode("iso-8859-1").decode('utf-8')

        #get info
        obj = re.compile(r'<div class="residential_item_name">小区物业</div>.*?item_value">(?P<property_company>.*?)</div>.*?residential_item_value">(?P<developer>.*?)</div>.*?residential_item_value">(?P<property_fee>.*?)</div>.*?<div class="attr_li_name">物业类型</div>.*?attr_li_value">(?P<house_type>.*?)</div>.*?<div class="attr_li_name">周边环境</div>.*?attr_li_value">(?P<surr_envi>.*?)</div>.*?attr_li_value">(?P<surr_school>.*?)</div>.*?attr_li_value">(?P<surr_trafic>.*?)</div>.*?attr_li_value">(?P<surr_business>.*?)</div>.*?attr_li_value">(?P<surr_entertain>.*?)</div>',re.S)
        result = obj.finditer(content)

        if os.path.exists('old_house_detail.csv'):
            w = open("old_house_detail.csv",mode="a+",newline='',encoding='utf-8')
            csvwriter = csv.writer(w)
        else:
            w = open("old_house_detail.csv",mode="w",newline='',encoding='utf-8') #在这个地方必须要加上utf-8！！
            csvwriter = csv.writer(w)
            header = ("property_company","developer","property_fee","house_type","surr_envi","surr_school","surr_trafic","surr_business","surr_entertain","id")
            csvwriter.writerow(header)

        for it in result:
            dic=it.groupdict() #把每个变量整合到一起变成dict

            dic["id"] = detail.iloc[row][7] 
            
            dic["surr_trafic"]=dic["surr_trafic"].replace("\n","")
            dic["surr_trafic"]=dic["surr_trafic"].replace("\r","")

            dic["surr_school"]=dic["surr_school"].replace("\n","")
            dic["surr_school"]=dic["surr_school"].replace("\r","")

            dic["surr_envi"]=dic["surr_envi"].replace("\n","")
            dic["surr_envi"]=dic["surr_envi"].replace("\r","")

            dic["surr_business"]=dic["surr_business"].replace("\n","")
            dic["surr_business"]=dic["surr_business"].replace("\r","")

            dic["surr_entertain"]=dic["surr_entertain"].replace("\n","")
            dic["surr_entertain"]=dic["surr_entertain"].replace("\r","")

            print(dic)

            csvwriter.writerow(dic.values())

    resp.close()

In [22]:
old_sub_detail()

<Response [200]>
{'property_company': '中山市明和物业管理有限公司', 'developer': '弘明房产有限公司', 'property_fee': '1.5 元/平米*月', 'house_type': '住宅', 'surr_envi': '银石雅园坐落于珠海最美丽的东西主干道，银石雅园东通吉大九洲港海滨西过立交桥可直接广州或珠海机场，南连拱北繁华闹市，被对圆明新圆旅游娱乐区。', 'surr_school': '十六小，十一中学', 'surr_trafic': '银石雅园巴士站，银石雅园南巴士站', 'surr_business': '富华里商业街，玖洲道公园购物街', 'surr_entertain': '圆明新园，梦幻水城，汉庭酒店，银石酒店，维也纳酒店', 'id': '1037'}
<Response [200]>
{'property_company': '深圳市金阳城物业管理', 'developer': '珠海正邦房地产开发有限公司', 'property_fee': '1.7 元/平米*月', 'house_type': '住宅', 'surr_envi': '口岸广场附近，沃尔玛商场', 'surr_school': '北岭小学，拱北小学，拱北口岸', 'surr_trafic': '1、2、10、11、13、21、32、33、35、62路公交车及观光巴士至北岭巴士站下车； 1路、2路、10路、10路、11路、32路、33路、35路、62路', 'surr_business': '迎宾广场，华润万家，口岸广场，莲花路万景城，MO商场，沃尔玛', 'surr_entertain': '星级酒店：银都酒店、粤海酒店、金都酒店、香江酒店，益健美食城', 'id': '1361'}
<Response [200]>
{'property_company': '华策物业', 'developer': '朝粤房地产有限公', 'property_fee': '2.2 元/平米*月', 'house_type': '住宅', 'surr_envi': 'KFC、麦当劳、拱北市场、文华书店、', 'surr_school': '夏湾小学/夏湾中学', 'surr_trafic': '35路、62路，到华骏大酒店站'

<Response [200]>
{'property_company': '富绅物业', 'developer': '富绅物业', 'property_fee': '1.2 元/平米*月', 'house_type': '住宅', 'surr_envi': '珠海市第二人民医院，桂花北邮局，中国银行、建设银行、交通银行，国美电器', 'surr_school': '拱北小学、拱北中学', 'surr_trafic': '4路路 36路路 62路路 82路，到新市花园站', 'surr_business': '保利来商业城(迎宾南路)、莲花万景城(莲花路)、拱北口岸购物广场、珠海口岸购物广场(北2门)', 'surr_entertain': '海兴海鲜大排档、红叶西餐厅、红蜻蜓美食、阿川酸辣粉、港兴茶餐厅', 'id': '206'}
<Response [200]>
{'property_company': '友谊物业', 'developer': '兰埔股份合作有限公司', 'property_fee': '1.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '非常不错，有圆明新园，梦幻水城', 'surr_school': '金钟小学，拱北中学，第十六中', 'surr_trafic': '62.69.13路到圆明山庄巴士站下', 'surr_business': '得一超市，华润万家，兰埔市场，八分田咖啡，兰埔美食街', 'surr_entertain': '圆明新园和梦幻水城', 'id': '278'}
<Response [200]>
{'property_company': '华发物业管理公司', 'developer': '华发实业股份有限公司', 'property_fee': '2.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '民医院、方华医院、仁和骨伤医院、传统医学研究所等；中国银行、工商银行、建设银行、交通银行、光大银行、东亚银行、渣打银行等', 'surr_school': '拱北小学 拱北中学', 'surr_trafic': '1路、2路、4路、8路、9路、10路、30路、31路、32路、33路、34路、608路，到万家百货站，或口岸地下广场站；', 

<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '272'}
<Response [200]>
{'property_company': '自家物业', 'developer': '自家物业', 'property_fee': '0.6 元/平米*月', 'house_type': '住宅', 'surr_envi': '国银行、建设银行、交通银行、珠海商业银行,\u3000医院：,\u3000珠海第二人民医院', 'surr_school': '拱北小学、拱北中学', 'surr_trafic': '北岭（1;2;10;10a;11;32;33;35;62', 'surr_business': '万家百货、米兰百货  迎宾广场、又一家超市、屈臣氏', 'surr_entertain': '莲花万景城(莲花路)、银都食街、珠海口岸购物广场', 'id': '238'}
<Response [200]>
{'property_company': '自家物业', 'developer': '自家物业', 'property_fee': '0.8 元/平米*月', 'house_type': '住宅', 'surr_envi': '板障山森林公园、文华书店、拱北市场', 'surr_school': '拱北小学、拱北中学；', 'surr_trafic': '1路、2路、观光02线、10路、10路A、11路、32路、33路、35路、62路，到北岭；', 'surr_business': '华润万家便利超市', 'surr_entertain': '新海利海鲜美食广场  香江酒店  谭鱼头珠海拱北店', 'id': '192'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.7 元/平米*月',

<Response [200]>
{'property_company': '鸿翔物业', 'developer': '珠海市汕业房地产开发有限公司', 'property_fee': '2.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '拱北最繁华的商业区，靠近拱北口岸，在口岸广场的左侧，是全珠海名副其实的地中之王。周边风景如画的环境，配上拱北口岸合四周的现代化建筑，尽显珠海的美丽风光', 'surr_school': '拱北小学、拱北中学', 'surr_trafic': '1、L1、2、2A、8、9、10、11、12、30、31、32、33、35、60、99、601、601B、608等，到万佳站', 'surr_business': '米兰百货、华润万家、迎宾广场、口岸地下广场；', 'surr_entertain': '莲花路步行街，侨光保健。', 'id': '231'}
<Response [200]>
{'property_company': '自家物业', 'developer': '自家物业', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '中国银行、建设银行、交通银行、珠海商业银行,\u3000医院：,\u3000珠海第二人民医院', 'surr_school': '北岭小学 拱北中学', 'surr_trafic': '北岭（1;2;10;10a;11;32;33;35;62;', 'surr_business': '万家百货、米兰百货  迎宾广场、又一家超市、屈臣氏', 'surr_entertain': '莲花万景城 (莲花路)、银都食街、珠海口岸购物广场', 'id': '284'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '1.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '195'}


<Response [200]>
{'property_company': '华发物业', 'developer': '宝润开发', 'property_fee': '1.8 元/平米*月', 'house_type': '住宅', 'surr_envi': '凌海名庭位于拱北生活旺地水湾酒巴街旁 配套非常完善 近关口 近生活旺地生活圈', 'surr_school': '拱北小学，拱北中学', 'surr_trafic': '4路; 9路; 13路;31路: 36路; 99路; 101路', 'surr_business': '莲花万景城、口岸地下购物广场、华润万佳等；', 'surr_entertain': '拱北酒吧街，莲花路步行街', 'id': '267'}
<Response [200]>
{'property_company': '自家物业', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '中国银行、建设银行、交通银行、珠海商业银行,\u3000医院：,\u3000珠海第二人民医院', 'surr_school': '北岭小学、拱北中学', 'surr_trafic': '', 'surr_business': '万家百货、米兰百货  迎宾广场、又一家超市、屈臣氏', 'surr_entertain': '莲花万景城 (莲花路)、银都食街、珠海口岸购物广场', 'id': '208'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '43283'}
<Response [200]>
{'property_company': '兰埔村委自营', 'developer': '兰埔股份合作公司', 'property_fee': '2.0 元/平米*月', 'house_

...

<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '靠近休闲广场，黄杨河边，空气好，休闲娱乐方便', 'surr_school': '井岸 实中 实小 学区', 'surr_trafic': 'k4.k5.601.609.501.502.503.504.301.303.304.306 411.401.402', 'surr_business': '得一超市，欣美超市，华润万家，中心市场，田洋市场', 'surr_entertain': '霞山公园，休闲广场，尖峰山公园，步行街，体育馆，西提公园，西提情侣路；', 'id': '3890'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '1.2 元/平米*月', 'house_type': '住宅', 'surr_envi': '封闭式花园小区，环境好，空气清新，安静', 'surr_school': '实中实小学区房', 'surr_trafic': '609路 601路 301路 306路 405路 504路', 'surr_business': '欣美超市，华润万家，顺客隆，步行购物街', 'surr_entertain': '尖峰山公园，步行街，体育馆，西提公园，西提情侣路；华发水郡湿地公园', 'id': '3908'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.8 元/平米*月', 'house_type': '住宅', 'surr_envi': '靠近霞山公园，环境优美，空气好。', 'surr_school': '二中二小学区', 'surr_trafic': 'k4.k5.601，609.501.502.503.504.301.303.304.306', 'surr_business': '鹏泰购物广场，得一超市，欣美超市，华润万家，中心市场，田洋市场', 'surr_entertain': '

<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '生活配套完善，环境优美。交通方便', 'surr_school': '井岸实验小学、井岸实验中学', 'surr_trafic': '308.307.601.601B.502.k6', 'surr_business': '得一超市、惠发百货、井岸步行街', 'surr_entertain': '西提公园，尖峰山公园', 'id': '4289'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '4607'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '4614'}
<Response [200]>
{'property_company': '没有物业', 'developer': '政府', 'property_fee': '0.0 元/平米*月', 'house_type': '地皮', 'surr_envi': '长途客运站', 'surr_school': '新环中心小学，白蕉新港幼儿园，田家炳中学', 'surr_trafic': '603', 'surr_business': '大洋百货商场、东盛百货', 'surr_entertain': '时代网吧', 'id'

<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '4702'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.6 元/平米*月', 'house_type': '住宅', 'surr_envi': '临近美平市场，小区临街旺铺，黄金地段', 'surr_school': '华丰小学学区', 'surr_trafic': '公交502', 'surr_business': '得一超市', 'surr_entertain': '', 'id': '1588'}
<Response [200]>
{'property_company': '', 'developer': '珠海市聚然贸易有限公司', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '4717'}
<Response [200]>
{'property_company': '森宇物业', 'developer': '森宇集团众智地产', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '熟生活社区，优质高中、初中、小学、幼儿园等环伺项目左右，周边有医院、市场、酒店，大型商业和特色商街，旅游度假资源环绕，品山、观海、享温泉，居家就像度假！', 'surr_school': '华丰小学，平沙中心幼儿园，平沙实验小学，平沙一中，珠海一中（平沙校区）', 'surr_trafic': '608路：拱北～海

<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '6922'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '1263'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '43298'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '43252'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr

<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '厂房', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '6320'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '', 'surr_school': '', 'surr_trafic': '', 'surr_business': '', 'surr_entertain': '', 'id': '3523'}
<Response [200]>
{'property_company': '万科', 'developer': '广拓置业', 'property_fee': '3.0 元/平米*月', 'house_type': '住宅', 'surr_envi': '亚洲风情园林，公园住宅范本，兼收园景河景，万科物业加持，八大商圈环伺，15年全龄段一站式教育全覆盖，咫尺珠海市第五人民医院', 'surr_school': '平沙实验学校（省一级学校）平沙一中（省一级学校）.华丰小学.汇华四季小学，平沙中心小学，华丰幼儿园.恒丰幼儿园', 'surr_trafic': '826-2路、812路、k30、Z142路等', 'surr_business': '京华MALL,京华家和花园商业街，时代港商荟，台湾风情街，平沙九号商业街，海泉华庭商业街，钰海国际广场，文德广场', 'surr_entertain': '海泉湾度假区，南国游艇俱乐部，平沙影视城， 荷包岛，飞沙滩，孖髻山公园，台创园', 'id': '43483'}
<Response [200]>
{'property_company': '', 'developer': '', 'property_fee': '0.0 元/平米*月', 'house_t

# crawler for old house type info

In [2]:
def sale(s_url,s_id,w):
    global s
    orientation = ["东","西","南","北","东南","东北","南北通","西南"]
    head={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
    }

    resp = requests.get(url=s_url,headers=head)
    print(resp)
    content = resp.text

    obj = re.compile(r'<div class="house_card">.*?<img src="(?P<photo>.*?)" alt=".*?<div class="text1">(?P<room_num>.*?)  ｜  (?P<area>.*?)  ｜  .*?<div class="price">(?P<price>.*?)<span class="unit">',re.S)
    result = obj.finditer(content)
    
    
    for it in result:
        dic=it.groupdict() #把每个变量整合到一起变成dict

        dic["price"]=dic["price"].replace("\n","")
        dic["price"]=dic["price"].replace("\r","")
        dic["price"] = "".join(filter(str.isalnum, dic["price"]))
        dic["price"] = dic["price"] + "万"
        
        dic["status"] = 0
        #in status 0 for sale
        
        dic["h_num"] = s
        s += 1
        dic["s_num"] = s_id
        dic["orientation"] = random.choice(orientation)
        
        w.writerow(dic.values())
        print(s)

In [3]:
def rent(r_url,s_id,w):
    global r
    orientation = ["东","西","南","北","东南","东北","南北通","西南"]
    head={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
    }

    resp = requests.get(url=r_url,headers=head)
    print(resp)
    content = resp.text

    obj = re.compile(r'<div class="house_card">.*?<img src="(?P<photo>.*?)" alt=".*?<div class="text1">(?P<room_num>.*?)  ｜  (?P<area>.*?)  </div>.*?<div class="price">(?P<price>.*?)<span class="unit">',re.S)
    result = obj.finditer(content)
    
    for it in result:
        dic=it.groupdict() #把每个变量整合到一起变成dict

        dic["price"]=dic["price"].replace("\n","")
        dic["price"]=dic["price"].replace("\r","")
        dic["price"] = "".join(filter(str.isalnum, dic["price"]))
        dic["price"] = dic["price"] + "元/月"
        
        dic["status"] = 1
        #in status 1 for rent
        
        dic["h_num"] = r
        r += 1
        dic["s_num"] = s_id
        dic["orientation"] = random.choice(orientation)

        w.writerow(dic.values())
        print(r)

In [4]:
def old_house_type():
    global s
    global r
    
    detail=pd.read_csv("old_house_info.csv",header=None)

    for row in range(1,2502):
        if os.path.exists('old_house_type_info.csv'):
            w = open("old_house_type_info.csv",mode="a+",newline='',encoding='utf-8')
            csvwriter = csv.writer(w)
        else:
            w = open("old_house_type_info.csv",mode="w",newline='',encoding='utf-8') #在这个地方必须要加上utf-8！！
            csvwriter = csv.writer(w)
            header = ("photo","room_num","area","price","status","h_num","s_num","orientation")
            csvwriter.writerow(header)
            
        head={
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
        }

        target1 = "https://www.okhome.com/sale?keyword=" + detail.iloc[row][3] + "&did=" + detail.iloc[row][7]
        resp1 = requests.get(url=target1,headers=head)
        print(resp1)
        content1 = resp1.text

        soup1 = BeautifulSoup(content1,"html.parser")

        for i in soup1.find_all(name="nav", attrs={"class" : "pagination"}):
            sub_url_sale=[target1]
            for j in i.find_all("a"):
                if(j.get('href') != None and j.text != "尾页"):
                    sub_url_sale.append("https://www.okhome.com/" + str(j.get('href')))

        s = 0
        for su in sub_url_sale:
            sale(su,detail.iloc[row][7],csvwriter)

        target2 = "https://www.okhome.com/rent?keyword=" + detail.iloc[row][3] + "&did=" + detail.iloc[row][7]
        resp2 = requests.get(url=target2,headers=head)
        print(resp2)
        content2 = resp2.text

        soup2 = BeautifulSoup(content2,"html.parser")

        for i in soup2.find_all(name="nav", attrs={"class" : "pagination"}):
            sub_url_rent=[target2]
            for j in i.find_all("a"):
                if(j.get('href') != None and j.text != "尾页"):
                    sub_url_rent.append("https://www.okhome.com/" + str(j.get('href')))

        r = 0
        for su in sub_url_rent:
            rent(su,detail.iloc[row][7],csvwriter)          
        

In [5]:
old_house_type()

<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
<Response [200]>
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
<Response [200]>
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
<Response [200]>
65
66
67
68
69
70
71
72
73
74
75
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
29
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
<Response [200]>
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
<Response [200]>
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
<Response [200]>
65
66
67
68
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
<Response [200]>
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
1

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
3
4
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
<Response [200]>
<Response [200]>
1
2
3
4
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
1
2
3
4
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
2
3
4
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
<Response [200]>
<Response [200]>
1
2
3
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
1
2
3
<Response [200]>
<Response [200]>
1
2
3
<R

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response 

<Response [200]>
33
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
29
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
28
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
21
22
23
24
25
26
27
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1

<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
<Response [200]>
17
18
19
20
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
8
9
10
11
12
13
14
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
<Response [200]>

...

<Response [200]>
1
2
3
4
5
6
7
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
5
6
<Response [200]>
<Response [200]>
1
2
3
4
5
6
7
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
3
4
5
<Response [200]

<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Re

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response 

<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
1
2
3
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
2
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
1
<Response [200]>
<Response [200]>
<Response [2